In [40]:
import aiosql
import sqlite3
import datetime as dt
import pprint as pp
# Connect to sqlite and enable auto commit
conn = sqlite3.connect("example.db", isolation_level=None)
conn.row_factory = sqlite3.Row

In [20]:
queries = aiosql.from_path("sql/", "sqlite3")

## Create Schema

Using the `#` operator you can execute sql statements as a script

> Note: Variable substitution is not possible 

In [3]:
help(queries.create_schema)

Help on method create_schema in module aiosql.queries:

create_schema(conn, *args, **kwargs) method of aiosql.queries.Queries instance
    Create table schema of users and blog in db



In [4]:
queries.create_schema(conn)

'DONE'

## Insert some data 

Let's insert a single user into the database using the `!` operator

In [52]:
queries.add_user(conn, **{"username": "mamamia2004", "firstname": "Nikolas", "lastname": "Zancher"})

Or insert a user and get the row id using the `<!` operator

In [53]:
userid = queries.add_user2(conn, **{"username": "papapia2005", "firstname": "Nicolae", "lastname": "Ionescu"})

print(userid)

4


Now lets add some blogs

In [54]:
blogid = queries.publish_blog2(conn, userid=userid, title="Hello World", content="Python is number one on TIOBE Index 🥳")

print(blogid)

8


Let's publish some blogs in bulk

In [16]:
blogs = [
    {"userid": 1, "title": "First Blog", "content": "...", "published": dt.datetime(2018, 1, 1)},
    {"userid": 1, "title": "Next Blog", "content": "...", "published": dt.datetime(2018, 1, 2)},
    {"userid": 2, "title": "Hey, Hey!", "content": "...", "published": dt.datetime(2018, 7, 28)},
    {"userid": 2, "title": "adipiscing fringilla", "content": "porttitor vulputate, posuere vulputate, lacus. Cras interdum.", "published": dt.datetime(2018, 7, 28)},
    {"userid": 2, "title": "adipiscing fringilla", "content": "porttitor vulputate, posuere vulputate, lacus. Cras interdum.", "published": dt.datetime(2018, 7, 28)},
    {"userid": 2, "title": "porttitor vulputate", "content": "posuere vulputate, lacus. Cras interdum.", "published": dt.datetime(2018, 7, 28)}
]

queries.bulk_publish(conn, blogs)

## Query some Data

Let's get the total count of users we can get the actual value using the `$`

In [55]:
queries.get_user_count(conn)

4

In [56]:
users = queries.get_users(conn)

pp.pprint(list(map(dict, users)), indent=2, compact=True, width=120)

[ {'firstname': 'Mike', 'lastname': 'Jones', 'userid': 1, 'username': 'badger77'},
  {'firstname': 'Micheal', 'lastname': 'Klandor', 'userid': 2, 'username': 'honeybadger77'},
  {'firstname': 'Nikolas', 'lastname': 'Zancher', 'userid': 3, 'username': 'mamamia2004'},
  {'firstname': 'Nicolae', 'lastname': 'Ionescu', 'userid': 4, 'username': 'papapia2005'}]


In [57]:
queries.get_blog_count(conn)

8

In [58]:
blogs = queries.get_blogs(conn)

pp.pprint(list(map(dict, blogs)), indent=2, width=120)

[ {'blogid': 1, 'content': 'blah blah.', 'published': '2021-10-11', 'title': 'Hi', 'userid': 2},
  {'blogid': 2, 'content': '...', 'published': '2018-01-01 00:00:00', 'title': 'First Blog', 'userid': 1},
  {'blogid': 3, 'content': '...', 'published': '2018-01-02 00:00:00', 'title': 'Next Blog', 'userid': 1},
  {'blogid': 4, 'content': '...', 'published': '2018-07-28 00:00:00', 'title': 'Hey, Hey!', 'userid': 2},
  { 'blogid': 5,
    'content': 'porttitor vulputate, posuere vulputate, lacus. Cras interdum.',
    'published': '2018-07-28 00:00:00',
    'title': 'adipiscing fringilla',
    'userid': 2},
  { 'blogid': 6,
    'content': 'porttitor vulputate, posuere vulputate, lacus. Cras interdum.',
    'published': '2018-07-28 00:00:00',
    'title': 'adipiscing fringilla',
    'userid': 2},
  { 'blogid': 7,
    'content': 'posuere vulputate, lacus. Cras interdum.',
    'published': '2018-07-28 00:00:00',
    'title': 'porttitor vulputate',
    'userid': 2},
  { 'blogid': 8,
    'content'

## Load and Run query from string

In [49]:
sql_str = """

-- name: get_user_blogs
-- Get blogs with a fancy formatted published date and author field
    select b.blogid,
           b.title,
           strftime('%Y-%m-%d %H:%M', b.published) as published,
           u.username as author
      from blogs b
inner join users u on b.userid = u.userid
     where u.username = :username
  order by b.published desc;
"""

qstr = aiosql.from_str(sql_str, "sqlite3")

In [59]:
user_blogs = qstr.get_user_blogs(conn, username="honeybadger77")
pp.pprint(list(map(dict, user_blogs)), indent=2, width=120)

[ {'author': 'honeybadger77', 'blogid': 1, 'published': '2021-10-11 00:00', 'title': 'Hi'},
  {'author': 'honeybadger77', 'blogid': 4, 'published': '2018-07-28 00:00', 'title': 'Hey, Hey!'},
  {'author': 'honeybadger77', 'blogid': 5, 'published': '2018-07-28 00:00', 'title': 'adipiscing fringilla'},
  {'author': 'honeybadger77', 'blogid': 6, 'published': '2018-07-28 00:00', 'title': 'adipiscing fringilla'},
  {'author': 'honeybadger77', 'blogid': 7, 'published': '2018-07-28 00:00', 'title': 'porttitor vulputate'}]
